In [1]:
#@title Import packages
!pip install -q pandas
!pip install -q altair
!pip install -q nba_api
!pip install -q plotly
import pandas as pd
import altair as alt
import plotly.graph_objects as go
import plotly.express as px

In [2]:
from nba_api.stats.endpoints import teamgamelog
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leagueleaders

tops = leagueleaders.LeagueLeaders(league_id='00', per_mode48='Totals', scope='S', season='2019-20',
                                   season_type_all_star='Regular Season', stat_category_abbreviation="REB")
top_ten_rb = pd.DataFrame(tops.get_normalized_dict()["LeagueLeaders"]).nsmallest(10, "RANK")
top_ten_rb = top_ten_rb.rename({'TEAM':'Team_ID'})

all_teams = pd.DataFrame(teams.get_teams())
all_teams = all_teams.rename(columns={"id":"Team_ID", 
                                      "full_name":"team_name",
                                      "abbreviation":"team_abbr"})[["Team_ID", "team_name", "team_abbr"]]

In [3]:
import time

all_games = pd.DataFrame() # Initialize empty data frame

for team_id in all_teams["Team_ID"]: # For each team in the league (30)
  current_game_log = teamgamelog.TeamGameLog(season='2019-20', season_type_all_star="Regular Season",
                                             team_id=team_id)
  current_df = pd.DataFrame(current_game_log.get_normalized_dict()['TeamGameLog'])
  
  current_df["game_num"] = list(pd.Series(current_df.index) + 1)[::-1] # Reverse game_num
  all_games = all_games.append(current_df) # Append this team's game data to the df of all teams
  time.sleep(0.5) # Sleep so I don't make the API mad at me

all_games = all_games.reset_index() # Reset the index so that it's easier to work with
all_games = all_games[['Team_ID','game_num','Game_ID','WL','GAME_DATE','MATCHUP',
 'W','L','W_PCT','MIN','FGM','FGA','FG_PCT','FG3M','FG3A','FG3_PCT','FTM',
 'FTA','FT_PCT','OREB','DREB','REB','AST','STL','BLK','TOV','PF','PTS']] # Rename the columns

all_games = all_teams.merge(all_games, how='inner', on="Team_ID") # merge data frames

ConnectionError: ('Connection aborted.', OSError("(10054, 'WSAECONNRESET')"))

In [ ]:
# Convert 'GAME_DATE' column to a datetime object so I can compare them
import datetime

def convert_to_date(date_string):
  return datetime.datetime.strptime(date_string, '%b %d, %Y')

all_games["GAME_DATE"] = all_games["GAME_DATE"].apply(convert_to_date)

In [ ]:
teams_to_show = ["MIL", "LAL", "CHA"]
graph_df = all_games[all_games["team_abbr"].isin(teams_to_show)] # filter by which teams you want to show
games_played = graph_df.groupby("team_abbr")["game_num"].max() # Find out how many games each team has played
game_limit = min(games_played) # Get min from each team's games played as limit for graph

px.scatter(graph_df,
        x="game_num",
        y="W_PCT",
        color="team_name",
        animation_frame="game_num",
        animation_group="team_abbr",
        title="Win Percentages for the Lakers, Bucks, and Hornets",
        range_x=[1, game_limit],
        range_y=[0,1],
        category_orders={"game_num":list(range(1,game_limit))})